# Pandas

## 계층적 인덱싱

In [2]:
# 한두 개보다 많은 키를 인덱스로 가지는 고차원 데이터를 저장하는 것이 필요
# Pandas는 기본적으로 3차원 4차원 데이터를 처리할 수 있는 Panel과 Panel4D 객체를 제공
# 하지만 실제로 더 일반적으로 사용되는 패턴은 단일 인덱스 내에 여러 인덱스 레벨을 포함하는 계층적 인덱싱(hierarchical indexing, 다중 인덱싱(multi-indexing))이라고 함
# 이 방식으로 고차원 데이터를 익숙한 1차원 Series와 2차원 DataFrame 객체로 간결하게 표현 가능

In [4]:
import pandas as pd
import numpy as np

## 다중 인덱스된 Series
+ 2차원 데이터 -> 1차원 Series에 표현

In [48]:
# 나쁜 방식

# 두 연도에 대해 미국 주의 데이터를 추적한다고 가정, Pandas 도구를 사용 간단하게 파이썬 튜플을 키 값으로 사용하려고 할 수 있음
# 이 다중 인덱스를 기반으로 시리즈를 인덱싱하거나 슬라이싱 할 수 있음
index = [('California', 2000),('California', 2010),
         ('New York', 2000),('New York', 2010),
         ('Texas', 2000),('Texas', 2010)]
population = [33871648, 37253956,
              18976457, 19378102,
              20851820, 25145561]
pop = pd.Series(population, index=index)
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

In [17]:
pop[('California', 2010):('Texas', 2000)]

(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
dtype: int64

In [18]:
pop[[i for i in pop.index if i[1] == 2010]] # 2010년의 모든 값을 선택해야 한다면 다소 지저분하고 느리기까지한 데이터 먼징(munging)을 해야 함

(California, 2010)    37253956
(New York, 2010)      19378102
(Texas, 2010)         25145561
dtype: int64

In [19]:
# 결론 : 원하는 결과를 내줌, 하지만 Pandas 슬라이싱 구문만큼 깔끔하지도 않고 대규모 데이터의 경우에는 효율적이지 않음

In [47]:
# 더 나은 방식: Pandas MultiIndex

# 튜플을 기반으로 한 인덱싱은 근본적으로 가장 기초적인 다중 인덱스
# Pandas의 MultiIndex 타입이 원하는 유형의 연산을 제공

In [21]:
index = pd.MultiIndex.from_tuples(index) # MultiIndex는 다중 레벨의 인덱싱을 포함, 이 경우 주 이름과 연도는 물론이고 이 레벨을 인코딩하는 각 데이터 점에 대한 여러 레이블을 갖고 있음
index

MultiIndex([('California', 2000),
            ('California', 2010),
            (  'New York', 2000),
            (  'New York', 2010),
            (     'Texas', 2000),
            (     'Texas', 2010)],
           )

In [23]:
pop = pop.reindex(index) # MultiIndex를 시리즈로 다시 인덱싱하면 데이터의 계층적 표현을 볼 수 있음
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [ ]:
# Series 표현의 첫 두 열은 다중 인덱스 값을 보여줌, 세 번째 열은 그 데이터를 보여줌
# 첫 번쨰 열의 항목 몇 개가 누락돼 있음
# 다중 인덱스 표현에서 빈 항목은 윗줄과 같은 값을 가리킴

In [24]:
pop[:, 2010] # 두 번째 인덱스가 2010인 모든 데이터에 접근, Pandas 슬라이싱 표기법 사용

California    37253956
New York      19378102
Texas         25145561
dtype: int64

In [25]:
# 결과 : 키 값 하나로 인덱스가 구성, 단순한 튜플 기반의 다중 인덱싱 해법보다 훨씬 더 편리하며 연산도 훨씬 더 효율적

In [46]:
# MultiIndex: 추가 차원

# 인덱스와 열 레이블을 가진 간단한 DataFrame을 사용, 동일한 데이터를 쉽게 저장할 수 있음
# Pandas는 이런 유사성을 염두에 두고 만들어짐
# unstack() 메서드는 다중 인덱스를 가진 Series를 전형적인 인덱스를 가진 DataFrame으로 빠르게 변환

In [27]:
pop_df = pop.unstack()
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


In [29]:
pop_df.stack() # stack() 메서드는 이와 반대되는 연산을 제공

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [30]:
# 계층적 인덱싱을 왜 알아야 할까? => 2차원 데이터를 1차원 Series에 표현하기 위해 다중 인덱싱을 사용할 수 있는 것처럼 
#                                 3차원이나 4차원 데이터를 Series나 DataFrame에 표현할 때도 사용할 수 있기 때문
# 다중 인덱스에서 각 추가 레벨은 데이터의 추가적인 차원을 표현
# 이 속성을 활용하면 표현할 수 있는 데이터 타임에 훨씬 더 많은 유연성을 제공

In [32]:
pop_df = pd.DataFrame({'total':pop,
                       'under18':[9267089, 9284094,
                                  4687374, 4318033,
                                  5906301, 6879014]})
# 연도별 각 주의 인구통계 데이터(18세 이하 인구수)를 별도의 열로 추가, MultiIndex를 이용하면 이 방식이 DataFrame에 열을 하나 추가하는 것만큼 쉬움
pop_df

total  under18
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010  25145561  6879014

In [33]:
# 모든 유니버설 함수와 다른 기능들도 계층적 인덱스와 잘 동작
# 위 데이터를 활용, 연도별로 18세 이하의 인구 비율을 계산
f_u18 = pop_df['under18'] / pop_df['total']
f_u18.unstack()

,2000,2010
California,0.273594,0.249211
New York,0.247010,0.222831
Texas,0.283251,0.273568


In [34]:
# 고차원 데이터도 빠르고 쉽게 가공하고 탐색 가능

## MultiIndex 생성 메서드

In [62]:
# 다중 인덱스를 가진 Series나 DataFrame을 생성하는 가장 간단한 방식은 생성자에 2개 이상의 인덱스 배열 리스트를 전달하는 것
df = pd.DataFrame(np.random.rand(4, 2),
                  index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                  columns=['data1', 'data2'])
df

data1     data2
a 1  0.045855  0.014836
  2  0.228579  0.360512
b 1  0.881530  0.941299
  2  0.701916  0.913719

In [55]:
# MultiIndx를 생성하는 작업은 백그라운드에서 일어남
# 적당한 튜플을 키로 갖는 딕셔너리를 전달하면 Pandas는 자동으로 이것을 인식해 기본 MultiIndex를 사용
data = {('California', 2000): 33871648,
        ('California', 2010): 37253956,
        ('Texas', 2000): 20851820,
        ('Texas', 2010): 25145561,
        ('New York', 2000): 18976457,
        ('New York', 2010): 19378102}
pd.Series(data)

California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
New York    2000    18976457
            2010    19378102
dtype: int64

In [56]:
# 명시적 MultiIndex 생성자

# 인덱스가 생성되는 방법에 더 많은 유연성을 제공하기 위해 pd.MultiIndex의 클래스 메서드 생성자를 사용
# ex) 각 레벨 내에 인덱스 값을 제공하는 간단한 배열 리스트로부터 MultiIndex를 생성 가능
pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1, 2, 1, 2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [57]:
pd.MultiIndex.from_tuples([('a',1), ('a',2), ('b',1), ('b',2)]) # 각 점의 여러 인덱스 값을 제공하는 튜플 리스트로부터 생성 가능

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [58]:
pd.MultiIndex.from_product([['a', 'b'], [1, 2]]) # 단일 인덱스의 데카르트 곱(Cartesian product)으로부터 MultiIndex를 생성 가능

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [59]:
# 이 외 : levels(각 레벨에서 사용할 수 있는 인덱스 값을 담고 있는 리스트의 리스트)와 labels(이 레이블을 참조하는 리스트의 리스트)를 전달함으로써 그 내부 인코딩을 사용해 직접 MultiIndex를 생성 가능
pd.MultiIndex(levels=[['a', 'b'], [1, 2]],
              codes=[[0, 0, 1, 1], [0, 1, 0, 1]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [60]:
# Series나 DataFrame을 생성할 때 index 인수로 이 객체를 기존 Series나 DataFrame의 reindex 메서드에 전달 가능

In [64]:
index = [('California', 2000),('California', 2010),
         ('New York', 2000),('New York', 2010),
         ('Texas', 2000),('Texas', 2010)]
population = [33871648, 37253956,
              18976457, 19378102,
              20851820, 25145561]
pop = pd.Series(population, index=index)
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

In [66]:
# MultiIndex 생성
multi_index = pd.MultiIndex.from_tuples(index, names=['state', 'year'])

# Series 생성
pop = pd.Series(population, index=multi_index)

In [67]:
# MultiIndex 레벨 이름

# MultiIndex의 레벨에 이름을 지정하는 것이 편리할 수 있음
# 위의 MultiIndex 생성자에 names 인수를 전달하거나 생성 후에 인덱스의 names 속성을 설정해 이름을 지정 가능
pop.index.names = ['state', 'year']
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [69]:
# 결과 : 관련 데이터세트가 많으면 이것이 다양한 인덱스 값의 의미를 파악할 수 있는 유용한 방식이 될 수 있음

In [70]:
# 열의 MultiIndex

# DataFrame에서 행과 열은 완전히 대칭적이며 행이 인덱스의 여러 레벨을 가질 수 있듯이 열도 여러 레벨을 가질 수 있음

In [74]:
# 가상 의료 데이터
# 계층적 인덱스와 열
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]],
                                   names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
                                     names=['subject', 'type'])

# 일부 데이터 모형 만들기
data = np.round(np.random.randn(4, 6), 1)
data[:, ::2] *= 10
data += 37

# DataFrame 생성하기
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      40.0  38.1  21.0  37.5  41.0  36.8
     2      38.0  38.2  24.0  37.3  17.0  37.9
2014 1      42.0  38.3  43.0  36.1  27.0  37.4
     2      50.0  35.2  56.0  37.0  40.0  35.5

In [75]:
# 행과 열 모두에 대한 멀티 인덱싱이 어떤 경우에 매우 유용하게 쓰이는지 알 수 있음
# 기본적으로 4차원 데이터, 여기서 차원은 대상(subject), 측정 유형(type), 연도(year), 방문 횟수(visit)
# ex) 사람 이름으로 최상위 열의 인덱스를 지정하고 그 사람의 정보를 포함하는 전체 DataFrame을 가져올 수 있음

In [76]:
health_data['Guido']

type          HR  Temp
year visit            
2013 1      21.0  37.5
     2      24.0  37.3
2014 1      43.0  36.1
     2      56.0  37.0

## MultiIndex 인덱싱 및 슬라이싱

In [77]:
# MultiIndex의 인덱싱과 슬라이싱은 직관적으로 설계, 인덱스를 추가된 차원으로 생각하면 이해 쉬움
# 다중 인덱스를 가진 Series 인덱싱 살펴본 후, 다중 인덱스를 가진 DataFrame을 살펴 봄

In [78]:
# 다중 인덱스를 가진 Series

# 주별 인구수 Series
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [79]:
pop['California', 2000] # 여러 용어로 인덱싱해서 단일 요소에 접근 가능

33871648

In [80]:
# MultiIndex는 부분 인덱싱(partial indexing)이나 인덱스 레벨 중 하나만 인덱싱하는 것도 지원
# 그 결과 더 낮은 수준의 인덱스를 유지하는 다른 Series를 얻게 됨
pop['California']

year
2000    33871648
2010    37253956
dtype: int64

In [81]:
# MultiIndex가 정렬돼 있다면 부분 슬라이싱도 가능
pop.loc['California':'New York']

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
dtype: int64

In [82]:
# 인덱스가 정렬돼 있다면 첫 번째 인덱스에 빈 슬라이스를 전달함으로써 더 낮은 레벨에서 부분 인덱싱을 수행 가능
pop[:, 2000]

state
California    33871648
New York      18976457
Texas         20851820
dtype: int64

In [83]:
# 다른 유형의 데이터 인덱싱과 선택 방식 역시 적용 가능, ex) 부울 마스크를 이용해 데이터를 선택 가능
pop[pop > 22000000]

state       year
California  2000    33871648
            2010    37253956
Texas       2010    25145561
dtype: int64

In [84]:
# 팬시 인덱싱(fancy Indexing)을 이용한 데이터 선택도 가능
pop[['California', 'Texas']]

state       year
California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
dtype: int64

In [85]:
# 다중 인덱스를 가진 DataFrame

# 다중 인덱스를 가진 DataFrame도 비슷한 방식으로 동작

In [86]:
# 의료 DataFrame
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      40.0  38.1  21.0  37.5  41.0  36.8
     2      38.0  38.2  24.0  37.3  17.0  37.9
2014 1      42.0  38.3  43.0  36.1  27.0  37.4
     2      50.0  35.2  56.0  37.0  40.0  35.5

In [87]:
health_data['Guido', 'HR']

year  visit
2013  1        21.0
      2        24.0
2014  1        43.0
      2        56.0
Name: (Guido, HR), dtype: float64

### 다중 인덱스 재정렬하기

In [88]:
# 정렬된 인덱스와 정렬되지 않은 인덱스

In [89]:
# 인덱스 스태킹 및 언스태킹

In [90]:
# 인덱스 설정 및 재설정 <-- 중요!

### 데이터세트 결합: Concat과 Append(없어짐)

In [91]:
# 두 개의 다른 데이터를 매우 간단하게 연결하는 것부터 데이터 간 겹치는 부분을 제대로 처리하는 복잡한 데이터베이스 스타일을 조인하고 병합하는 것까지 다양하게 사용
# Series와 DataFrame은 이 유형의 연산을 염두에 두고 만들어진 것
# Pandas는 이러한 유형의 데이터 랭글링(data wrangling)을 빠르고 간단하게 할 수 있는 함수와 메서드를 제공

In [92]:
import pandas as pd
import numpy as np

In [93]:
def make_df(cols, ind):
    """빠르게 DataFrame 생성"""
    data = {c: [str(c) + str(i) for i in ind]
            for c in cols}
    return pd.DataFrame(data, ind)

# DataFrame 예제
make_df('ABC', range(3))

,A,B,C
0,A0,B0,C0
1,A1,B1,C1
2,A2,B2,C2


In [94]:
# 여러 개의 데이터프레임을 나란히 표시할 수 있는 간단한 클래스
# IPython/Jupyter가 객체를 표시하는 데 사용하는 특수 _repr_html_ 메서드를 사용
class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;" >"""
    def __init__(self, *args):
        self.args = args
    def _repr_html(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
            for a in self.args)
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
            for a in self.args)

### 복습: NumPy 배열 연결

In [95]:
# Series와 DataFrame 객체의 연결은 np.concatenate 함수를 사용하는 NumPy 배열 연결과 매우 유사
# 이 함수를 이용하면 두 개 이상의 배열의 콘텐츠를 하나의 배열로 결합할 수 있음
x = [1, 2, 3]
y = [4, 5, 6]
z = [7, 8, 9]
np.concatenate([x, y, z])

array([1, 2, 3, 4, 5, 6, 7, 8, 9])

In [96]:
# 첫 번째 인수는 연결할 배열의 리스트나 튜플, axis 키워드를 사용해 결과를 어느 축에 따라 연결할 것인지 지정 가능
x = [[1, 2],
     [3, 4]]
np.concatenate([x, x], axis=1)

array([[1, 2, 1, 2],
       [3, 4, 3, 4]])

### pd.concat을 이용한 간단한 연결

In [97]:
# Pandas에는 np.concateante와 구문이 매우 비슷하지만 다양한 옵션을 가진 pd.concat() 함수가 있음

In [99]:
# Pandas 0.19 버전에서 pd.concat() 함수 시그니처
#pd.concat(objs, axis=0, join='outer', join_axes=None, ignore_index=False,
#         keys=None, levels=None, names=None, verify_integrity=False,
#          sort=False, copy=True)

In [100]:
# np.concatenate()를 배열을 간단하게 연결하는 데 사용할 수 있는 것처럼 pd.concat()은 Series나 DataFrame 객체를 간단하게 연결할 때 사용
ser1 = pd.Series(['A', 'B', 'C'], index=[1, 2, 3])
ser2 = pd.Series(['D', 'E', 'F'], index=[4, 5, 6])
pd.concat([ser1, ser2])

1    A
2    B
3    C
4    D
5    E
6    F
dtype: object

In [107]:
# pd.concat()을 이용하면 DataFrame 같은 고차원 객체를 연결 가능
df1 = make_df('AB', [1, 2])
df2 = make_df('AB', [3, 4])
print(df1.to_string() + "\n"); print(df2.to_string() + "\n"); print(pd.concat([df1, df2]).to_string())

    A   B
1  A1  B1
2  A2  B2

    A   B
3  A3  B3
4  A4  B4

    A   B
1  A1  B1
2  A2  B2
3  A3  B3
4  A4  B4


In [108]:
# 인덱스 복제

# np.concatenate와 pd.concat의 중요한 차이 : Pandas에서의 연결은 그 결과가 복제된 인덱스를 가지더라도 인덱스를 유지한다는 데 있음
x = make_df('AB', [0, 1])
y = make_df('AB', [2, 3])
y.index = x.index # 복제 인덱스 생성!
print(x.to_string() + "\n"); print(y.to_string() + "\n"); print(pd.concat([x, y]).to_string())

    A   B
0  A0  B0
1  A1  B1

    A   B
0  A2  B2
1  A3  B3

    A   B
0  A0  B0
1  A1  B1
0  A2  B2
1  A3  B3


In [109]:
# 결과 : 인덱스가 반복, DataFrame 내에서는 유효하지만 결과가 바람직하지 않은 경우가 있음 -> pd.concat()은 이 문제를 처리하는 몇 가지 방법을 제공

In [110]:
# 1. 반복을 에러로 잡아낸다

# pd.concat()의 결과에서 인덱스가 겹치지 않는지 간단히 검증하고 싶으면 verify_integrity 플래그를 지정
# 이 플래그를 True로 설정하면 연결 작업에서 중복 인덱스가 있을 때 예외가 발생

In [113]:
# 확인을 위해 오류를 잡아내고 메시지를 출력
try:
    pd.concat([x, y], verify_integrity=True)
except ValueError as e:
    print("ValueError:", e)

ValueError: Indexes have overlapping values: Index([0, 1], dtype='int64')


In [117]:
# 2. 인덱스를 무시한다

# 인덱스 자체가 중요하지 않은 경우에는 그냥 인덱스를 무시, ignore_index 플래그를 사용해 이 옵션을 지정 가능
# 이 플래그를 True로 설정하면 연결 작업은 결과 Series에 새로운 정수 인덱스를 생성
print(x.to_string() + "\n"); print(y.to_string() + "\n"); print(pd.concat([x, y], ignore_index=True).to_string())

    A   B
0  A0  B0
1  A1  B1

    A   B
0  A2  B2
1  A3  B3

    A   B
0  A0  B0
1  A1  B1
2  A2  B2
3  A3  B3


In [118]:
# 3. 다중 인덱스 키를 추가한다



In [119]:
# 조인을 이용한 연결

In [120]:
# 데이터세트 결합하기: 병합과 조인

In [122]:
import pandas as pd
import numpy as np

class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}{1}"""
    def __init__(self, *args):
        self.args = args
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

### 관계 대수

### 조인 작업의 분류

In [123]:
#일대일 조인

df1 = pd.DataFrame()

### 예제: 미국 주 데이터

In [125]:
pop = pd.read_csv('data/state-population.csv')
areas = pd.read_csv('data/state-areas.csv')
abbrevs = pd.read_csv('data/state-abbrevs.csv')

print(pop.head().to_string() + "\n"); print(areas.head().to_string() + "\n"); print(abbrevs.head());

  state/region     ages  year  population
0           AL  under18  2012   1117489.0
1           AL    total  2012   4817528.0
2           AL  under18  2010   1130966.0
3           AL    total  2010   4785570.0
4           AL  under18  2011   1125763.0

        state  area (sq. mi)
0     Alabama          52423
1      Alaska         656425
2     Arizona         114006
3    Arkansas          53182
4  California         163707

        state abbreviation
0     Alabama           AL
1      Alaska           AK
2     Arizona           AZ
3    Arkansas           AR
4  California           CA


In [126]:
abbrevs.head()

,state,abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


In [129]:
merged = pd.merge(pop, abbrevs, how='outer', left_on='state/region', right_on='abbreviation')
merged = merged.drop('abbreviation', axis=1) # 중복 정보 삭제
merged.head()

,state/region,ages,year,population,state
0,AK,total,1990,553290.0,Alaska
1,AK,under18,1990,177502.0,Alaska
2,AK,total,1992,588736.0,Alaska
3,AK,under18,1991,182180.0,Alaska
4,AK,under18,1992,184878.0,Alaska


In [130]:
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

In [132]:
merged[merged['population'].isnull()].head()

,state/region,ages,year,population,state
1872,PR,under18,1990,NaN,NaN
1873,PR,total,1990,NaN,NaN
1874,PR,total,1991,NaN,NaN
1875,PR,under18,1991,NaN,NaN
1876,PR,total,1993,NaN,NaN


In [133]:
merged.loc[merged.state.isnull(), 'state/region'].unique()

array(['PR', 'USA'], dtype=object)

In [136]:
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico'
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States'
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

In [137]:
final = pd.merge(merged, areas, on='state', how='left')
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AK,total,1990,553290.0,Alaska,656425.0
1,AK,under18,1990,177502.0,Alaska,656425.0
2,AK,total,1992,588736.0,Alaska,656425.0
3,AK,under18,1991,182180.0,Alaska,656425.0
4,AK,under18,1992,184878.0,Alaska,656425.0


In [138]:
final.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

In [140]:
final['state'][final['area (sq. mi)'].isnull()].unique()

array(['United States'], dtype=object)

In [141]:
final.dropna(inplace=True)
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AK,total,1990,553290.0,Alaska,656425.0
1,AK,under18,1990,177502.0,Alaska,656425.0
2,AK,total,1992,588736.0,Alaska,656425.0
3,AK,under18,1991,182180.0,Alaska,656425.0
4,AK,under18,1992,184878.0,Alaska,656425.0


In [152]:
data2010 = final.query("year == 2010 & ages == 'total'")
data2010.head()

,state/region,ages,year,population,state,area (sq. mi)
43,AK,total,2010,713868.0,Alaska,656425.0
51,AL,total,2010,4785570.0,Alabama,52423.0
141,AR,total,2010,2922280.0,Arkansas,53182.0
149,AZ,total,2010,6408790.0,Arizona,114006.0
197,CA,total,2010,37333601.0,California,163707.0


In [153]:
data2010.set_index('state', inplace=True)
density = data2010['population'] / data2010['area (sq. mi)']

In [154]:
density.sort_values(ascending=False, inplace=True)
density.head()

state
District of Columbia    8898.897059
Puerto Rico             1058.665149
New Jersey              1009.253268
Rhode Island             681.339159
Connecticut              645.600649
dtype: float64

In [155]:
density.tail()

state
South Dakota    10.583512
North Dakota     9.537565
Montana          6.736171
Wyoming          5.768079
Alaska           1.087509
dtype: float64

In [163]:
pop.head().shape, areas.head().shape

((5, 4), (5, 2))

In [165]:
merged.head().shape

(5, 5)

In [169]:
merged['state/region'].unique(), len(pop['state/region'].unique())

(array(['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
        'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
        'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
        'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'PR', 'RI', 'SC', 'SD', 'TN',
        'TX', 'USA', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY'],
       dtype=object),
 53)

In [170]:
merged.shape

(2544, 5)

In [171]:
merged.head()

,state/region,ages,year,population,state
0,AK,total,1990,553290.0,Alaska
1,AK,under18,1990,177502.0,Alaska
2,AK,total,1992,588736.0,Alaska
3,AK,under18,1991,182180.0,Alaska
4,AK,under18,1992,184878.0,Alaska


### GroupBy: 분할, 적용, 결합

In [ ]:
# 간단한 집계 : 데이터세트의 전반적인 특성을 알려줌,
# 조건부 집계 : groupby 연산으로 구현
# SQL 데이터베이서 언어에서 유래, R 분석 권위자 해들리 위컴(Hadley Wickham)이 최초로 고안한 용어 분할(split), 적용(apply), 결합(comvine)

In [173]:
# 분할, 적용(요약 집계), 결합

# -분할 : 지정된 키 값을 기준으로 DataFrame을 나누고 분류하는 단계
# -적용 : 개별 그룹 내에서 일반적으로 집계, 변환, 필터링 같은 함수를 계산
# -결합 : 이 연산의 결과를 결과 배열에 병합

In [175]:
df = pd.DataFrame({'key' : ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data' : range(6)}, columns = ['key', 'data'])
df

,key,data
0,A,0
1,B,1
2,C,2
3,A,3
4,B,4
5,C,5


In [178]:
df.groupby('key') # DataFrame의 groupby() 메서드에 원하는 키 열의 이름을 전달해 가장 기본적인 분할-적용-결합 연산을 계산

In [181]:
# DataFrame의 집합이 아니라 DataFrameGroupBy 객체가 반환

df.groupby('key').sum()

,data
key,
A,3
B,5
C,7


In [193]:
pip install seaborn

Note: you may need to restart the kernel to use updated packages.


In [3]:
import seaborn as sns
df = sns.load_dataset('planets')
df.shape

(1035, 6)

In [11]:
df

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300000,7.10,77.40,2006
1,Radial Velocity,1,874.774000,2.21,56.95,2008
2,Radial Velocity,1,763.000000,2.60,19.84,2011
3,Radial Velocity,1,326.030000,19.40,110.62,2007
4,Radial Velocity,1,516.220000,10.50,119.47,2009
...,...,...,...,...,...,...
1030,Transit,1,3.941507,NaN,172.00,2006
1031,Transit,1,2.615864,NaN,148.00,2007
1032,Transit,1,3.191524,NaN,174.00,2007
1033,Transit,1,4.125083,NaN,293.00,2008


In [5]:
df.dropna().describe()

,number,orbital_period,mass,distance,year
count,498.00000,498.000000,498.000000,498.000000,498.000000
mean,1.73494,835.778671,2.509320,52.068213,2007.377510
std,1.17572,1469.128259,3.636274,46.596041,4.167284
min,1.00000,1.328300,0.003600,1.350000,1989.000000
25%,1.00000,38.272250,0.212500,24.497500,2005.000000
50%,1.00000,357.000000,1.245000,39.940000,2009.000000
75%,2.00000,999.600000,2.867500,59.332500,2011.000000
max,6.00000,17337.500000,25.000000,354.000000,2014.000000


In [6]:
# GroupBy 객체

df.groupby('method')

In [7]:
df.groupby('method')['orbital_period']

In [8]:
df.groupby('method')['orbital_period'].median() # 궤도 주기(일 단위)

method
Astrometry                         631.180000
Eclipse Timing Variations         4343.500000
Imaging                          27500.000000
Microlensing                      3300.000000
Orbital Brightness Modulation        0.342887
Pulsar Timing                       66.541900
Pulsation Timing Variations       1170.000000
Radial Velocity                    360.200000
Transit                              5.714932
Transit Timing Variations           57.011000
Name: orbital_period, dtype: float64

In [12]:
result = df.groupby(['method'])['orbital_period'].mean() # 궤도 주기(일 단위)
result

method
Astrometry                          631.180000
Eclipse Timing Variations          4751.644444
Imaging                          118247.737500
Microlensing                       3153.571429
Orbital Brightness Modulation         0.709307
Pulsar Timing                      7343.021201
Pulsation Timing Variations        1170.000000
Radial Velocity                     823.354680
Transit                              21.102073
Transit Timing Variations            79.783500
Name: orbital_period, dtype: float64

In [10]:
# 그룹 내 반복

# GroupBy 객체는 그룹을 직접 순회할 수 있도록 지원, 각 그룹을 Series나 DataFrame으로 반환
for (method, group) in df.groupby('method'):
    print("{0:30s} shape={1}".format(method, group.shape))

Astrometry                     shape=(2, 6)
Eclipse Timing Variations      shape=(9, 6)
Imaging                        shape=(38, 6)
Microlensing                   shape=(23, 6)
Orbital Brightness Modulation  shape=(3, 6)
Pulsar Timing                  shape=(5, 6)
Pulsation Timing Variations    shape=(1, 6)
Radial Velocity                shape=(553, 6)
Transit                        shape=(397, 6)
Transit Timing Variations      shape=(4, 6)


In [ ]:
# 디스패치 메서드(Dispatch Method)

# GroupBy 객체가 명시적으로 구현하지 않은 메서드는 그것이 DataFrame 객체든 Series 객체든 상관 없이 